<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/MLGymGALunar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI]((https://gym.openai.com)

Voir la page d'[introduction à Gym](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLIntroGym.ipynb).

### Installation de gym

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [ ]:
!pip install gym
!pip install Box2D


In [5]:
#GYM
import gym
import Box2D as b2

#Random
from random import  randint
#
from math import exp
#for sorting
import functools
#array
import numpy as np

#for charts
import matplotlib.pyplot as plt
#On MAc : 
#import matplotlib
#matplotlib.use("MacOSX")

#### <font color=red>Sous colab</font>
*Sous colab, l'affichage ne sera pas dynamique.. Il faudra passer soit par un rendu sous forme de plot (matplotlib), soit passer par une vidéo à enregistrer...*

***Solution pour affichage statique***<br>
*Il vous faut importer, seulement si vous utilisez colab, les packages suivants :*

In [3]:
!sudo apt-get update
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Password:
sudo: a password is required
^C


In [3]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (56.0.0)


In [ ]:
#GYM
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [2]:
#GYM
import gym
import Box2D as b2

#Random
from random import  randint
#
from math import exp
#for sorting
import functools
#array
import numpy as np

#for charts
import matplotlib.pyplot as plt


---
## L'environnement LunarLander
On se base ici sur l'environnement `LunarLander-v2`.

### Charger la version originale de LunarLander
On utilisera une version modifiée de l'environnement. Mais pour charger la version de base, sous tout outil autre que colab, on utilise la commande suivante : 

In [7]:
env = gym.make('LunarLander-v2') 

Mais **<font color=red>sous colab</font>** on utilise la commande

In [3]:
env = wrap_env(gym.make("LunarLander-v2"))

---
### Les valeurs des caractéristiques
Pour `LunarLander-v2`, vous trouverez l'explication des valeurs [ici](https://gym.openai.com/envs/LunarLander-v2/).

Il s'agit de faire atterrir une sonde sur une planète.

L'espace des actions est une valeur discrète dans l'intervalle [0,3] : 
  - 0 : Ne rien faire
  - 1 : allumage du moteur gauche
  - 2 : allumage du moteur central
  - 3 : allumage du moteur droit

Les observations sont au nombre de 8 : 
  - obs[0] : coordonnée horizontale
  - obs[1] : coordonnée verticale
  - obs[2] : vitesse horizontale
  - obs[3] : vitesse verticale
  - obs[4] : angle de la sonde
  - obs[5] : vitesse angulaire (de rotation)
  - obs[6] : 1 si le pied gauche de la sonde est au contact du sol, 0 sinon
  - obs[7] : 1 si le pied droit de la sonde est au contact du sol, 0 sinon

Le but est donc de faire atterrir la sonde dans une position donnée (0,0) avec une vitesse suffisamment faible pour éviter le crash.


In [4]:
print(env.spec)
print("nombre et type d'actions = ", env.action_space) 
print("nombre et type d'observations = ", env.observation_space) 
print("valeurs de  observations les plus basses = ", env.observation_space.low) 
print("valeurs de observations les plus elevées = ", env.observation_space.high) 

EnvSpec(LunarLander-v2)
nombre et type d'actions =  Discrete(4)
nombre et type d'observations =  Box(-inf, inf, (8,), float32)
valeurs de  observations les plus basses =  [-inf -inf -inf -inf -inf -inf -inf -inf]
valeurs de observations les plus elevées =  [inf inf inf inf inf inf inf inf]


___
### Comportement non déterministe !
`env.reset()` initialise les variables d'environnement. L'exécution est obligatoire après avoir chargé l'environnement. Ici ce sont les coordonnées de départ, l'angle d'attaque et les vitesses ont des valeurs tirées aléatoirement dans des intervales pré-définis. De plus les accélérations demandées ax moteurs sont elles aussi soumises à des variations.

C'est donc un environnement non déterministe par défaut, `env.reset()` retourne des valeurs légèrement différentes à chaque exécution : 

In [29]:
env.reset()

array([-0.00364351,  1.4210072 , -0.36906666,  0.4482938 ,  0.00422874,
        0.08359913,  0.        ,  0.        ], dtype=float32)

**<font color=red>L'algorithme génétique se prète mal aux environnement non stables</font>**; aussi dans cet exercice, **il faudra utiliser le code https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/lunarsourcegym.py que vous chargerez dans le même répertoire que votre code.** 

Cette version est une reprise du code de [Gym.OpenAI]((https://gym.openai.com), mais qui fixe les valeurs initiales du LunarLander et qui ôte les perturbations aléatoires lors des accélérations moteurs.

Une fois le code chargé, pour utiliser l'environnement, il faut utiliser le code suivant : 

In [5]:
from lunarlanderdeterministe import MonLunarLander

env = MonLunarLander()
# sous collab, on écrira : 
# env = wrap_env(MonLunarLander())

On utilisera tout de même une fonction `reset()` pour fixer les angles et vitesses initiales : 

In [ ]:
def reset():
    """reinitialisation de l'environnement monlunarlander et mise en place de l'angle d'attaque"""
    env.reset();
    env.lander.angle=0
    env.lander.angularVelocity = 0
    env.lander.ApplyForceToCenter( (0,0,), True)
    env.lander.linearVelocity = b2.b2Vec2()

---
### Les actions
- Dans `LunarLander-v2`, comme indiqué plus haut, 0 n'allume aucun, 1 allume bièvement le moteur gauche, 2 allume bièvement le moteur central 3 allume bièvement le moteur droit.

- ***Exécuter une action :***
  - *exécution* : `observation, reward,done,info = env.step(action)`
    - `observation`: ici les 8 éléments d'observation
    - `reward`: utilité de l'état (récompense 100 si bien posé, -100 si crash)
    - `done` : booléen (True si but (position) atteinte)
    - `info` : éventuelle information retournée par l'outil

**Exemples d'actions**

In [15]:
def test_action(no: int, nb: int = 1):
    """effectue nb fois l'action no dans l'environnement chargé"""
    done:bool = False
    i:int = 0
    while i<nb and not done:
        i += 1
        env.render()
        obs, reward, done, _ = env.step(no)
        print(reward, done)
    return done

def test():
  done = test_action(2, 40)
  if (not done): done = test_action(0, 40)
  if (not done): done = test_action(1, 40)
  if (not done): done = test_action(3, 40)

In [ ]:
# sous collab : 
# env = wrap_env(MonLunarLander())
reset()
test()



**Sous Colab**


In [ ]:
env.close()
show_video()

ATTENTION, sous colab, bien répéter la ligne
`env = wrap_env(MonLunarLander())' avant le reset et la création de vidéos


---

# Test d'algorithme génétique
## Créer un programme évolutionnaire pour atteindre l'objectif
- Utiliser l'environnement `LunarLander-v2`, car il est plus rapide 
- Programmez (en python) un algo génétique avec initialement les éléments suivants : 
  - sequence : tableau de valeurs entières 0, 1, 2 ou 3.
  - 1 point de croisement (au centre)
    - *(pour commencer, on pourrait proposer 2 points de croisement (au 1/3, et 2/3)), ou plus*
  - taille sequence : taille_seq = 100
  - taille population initiale : taille_pop = 100
  - taux de gènes mutants dans une séquence : taux_mut_seq = 0.1
  - taux de séquences mutantes dans la population : taux_mut_pop = 0.2
  - nombre max de cycles de reproductions : nb_cycles = 3000


- Pour tester une séquence, il suffit de balayer le tableau et de lancer l'action correspondante..
- Les 4 meilleures séquences (11,22,33,44) se reproduisent entre elles, donnant donc 12 fils : 12, 13, 14, 21, 23, 24, 31, 32, 34, 41, 42, 43.
- Idéalement, un filtre doit permettre de supprimer les séquences identiques qui pourraient venir des croisements et des mutations.

- Tester votre algo sur le nb de générations et afficher le rendu toutes les 100 générations et bien sûr le rendu de la meilleure séquence finale.
  - (uniquement la position finale *colab*)
  - (sous forme d'animation sous autre outil)


- ***NB:*** 
  - Au plus simple il est possible d'utiliser une population (liste) de couples [sequence, utilite] où l'utilité serait la distance en x.

---
### AIDE
  - Pour commencer, on pourra se limiter aux actions 0 & 2.. Les séquences ne contiennent donc que des chiffres 0 & 2 placés aléatoirement au départ.
  - **Pour évaluer une séquence**, on peut se baser sur le reward calculés par l'environnement. Mais il est plutôt recommandé de créer sa propre fonction d'évaluation. 
    - La zone d'arrivée se situe entre les drapeaux. Ces drapeaux ont pour coordonnées x respectivement -14 et +14. On peut aussi laisser une certaine marge de manoeuvre en y, dù à la hauteur de la sonde. ainsi une arrivée en y = +-5 entre les drapeaux pourra être considérée comme satisfaisante. La vitesse verticale doit être faible (<1 m/s). Et les "pattes" devront avoir été déployées.
  - **Pour le croisement**, il est possible de choisir aléatoirement 4 séquences dans la population plutôt que les 4 premières.. Ou pseudo-aléatoirement, avec une plus grande chance d'être sélectionnée pour les meilleures séquences..


---
### V2 : Arrivée avec angle 
- Maintenant la sonde arrive avec un angle donné (0.3 par exemple) et une vitesse angulaire de 0.05..
Il faut bien sûr maintenant activer les moteurs gauches et droits. Donc les séquences contiennent les chiffres 0 à 3.
  - une mutation devra être douce.. Ainsi, à partir des actions 0 & 2, il est possible de muter vers les actions 0 à 3.
  - mais à partir d'une actions 1 (respectivement 3), il n'est possible que de muter vers les actions 0,1,2 (respectivement 2,3,0).
- Vérifiez que votre algorithme trouve bien la solution... 

In [ ]:
def reset2(angle, vitesse_angulaire):
    env.reset();
    env.lander.angle = angle
    env.lander.angularVelocity = 0
    env.lander.ApplyForceToCenter( (
            0,0,
            ), False)
    env.lander.linearVelocity = b2.b2Vec2()

In [ ]:
reset2(0.3, 0.05)
#application de l'algo...

----
### BONUS
Cherchez les séquences les plus courtes pour les deux cas de figure..